# Implementation in TensorFlow

In this section, we will show you how to implement a fully connected network in TensorFlow. We won’t need to introduce many new TensorFlow primitives in this section since we have already covered most of the required basics.

# Installing DeepChem

In this section, you will use the DeepChem machine learning toolchain for your experiments (full disclosure: one of the authors was the creator of DeepChem). Detailed installation directions for DeepChem can be found online, but briefly the Anaconda installation via the conda tool will likely be most convenient.

# Tox21 Dataset

For our modeling case study, we will use a chemical dataset. Toxicologists are very interested in the task of using machine learning to predict whether a given compound will be toxic or not. This task is extremely complicated, since today’s science has only a limited understanding of the metabolic processes that happen in a human body. However, biologists and chemists have worked out a limited set of experiments that provide indications of toxicity. If a compound is a “hit” in one of these experiments, it will likely be toxic for a human to ingest. However, these experiments are often costly to run, so data scientists aim to build machine learning models that can predict the outcomes of these experiments on new molecules.

One of the most important toxicological dataset collections is called Tox21. It was released by the NIH and EPA as part of a data science initiative and was used as the dataset in a model building challenge. The winner of this challenge used multitask fully connected networks (a variant of fully connected networks where each network predicts multiple quantities for each datapoint). We will analyze one of the datasets from the Tox21 collection. This dataset consists of a set of 10,000 molecules tested for interaction with the androgen receptor. The data science challenge is to predict whether new molecules will interact with the androgen receptor.

Processing this dataset can be tricky, so we will make use of the MoleculeNet dataset collection curated as part of DeepChem. Each molecule in Tox21 is processed into a bit-vector of length 1024 by DeepChem. Loading the dataset is then a few simple calls into DeepChem (Example 4-1).

Example 4-1. Load the Tox21 dataset

In [ ]:
import deepchem as dc
_, (train, valid, test), _ = dc.molnet.load_tox21()
train_X, train_y, train_w = train.X, train.y, train.w
valid_X, valid_y, valid_w = valid.X, valid.y, valid.w
test_X, test_y, test_w = test.X, test.y, test.w

Here the X variables hold processed feature vectors, y holds labels, and w holds example weights. The labels are binary 1/0 for compounds that interact or don’t interact with the androgen receptor. Tox21 holds imbalanced datasets, where there are far fewer positive examples than negative examples. w holds recommended per-example weights that give more emphasis to positive examples (increasing the importance of rare examples is a common technique for handling imbalanced datasets). We won’t use these weights during training for simplicity. All of these variables are NumPy arrays.

Tox21 has more datasets than we will analyze here, so we need to remove the labels associated with these extra datasets (Example 4-2).

Example 4-2. Remove extra datasets from Tox21

In [ ]:
# Remove extra tasks
train_y = train_y[:, 0]
valid_y = valid_y[:, 0]
test_y = test_y[:, 0]
train_w = train_w[:, 0]
valid_w = valid_w[:, 0]
test_w = test_w[:, 0]

# Accepting Minibatches of Placeholders

In the previous chapters, we created placeholders that accepted arguments of fixed size. When dealing with minibatched data, it is often convenient to be able to feed batches of variable size. Suppose that a dataset has 947 elements. Then with a minibatch size of 50, the last batch will have 47 elements. This would cause the code in Chapter 3 to crash. Luckily, TensorFlow has a simple fix to the situation: using None as a dimensional argument to a placeholder allows the placeholder to accept tensors with arbitrary size in that dimension (Example 4-3).

Example 4-3. Defining placeholders that accept minibatches of different sizes

In [ ]:
d = 1024
with tf.name_scope("placeholders"):
  x = tf.placeholder(tf.float32, (None, d))
  y = tf.placeholder(tf.float32, (None,))

Note d is 1024, the dimensionality of our feature vectors.

# Implementing a Hidden Layer

The code to implement a hidden layer is very similar to code we’ve seen in the last chapter for implementing logistic regression, as shown in Example 4-4.

Example 4-4. Defining a hidden layer

In [ ]:
with tf.name_scope("hidden-layer"):
  W = tf.Variable(tf.random_normal((d, n_hidden)))
  b = tf.Variable(tf.random_normal((n_hidden,)))
  x_hidden = tf.nn.relu(tf.matmul(x, W) + b)

We use a tf.name_scope to group together introduced variables. Note that we use the matricial form of the fully connected layer. We use the form xW instead of Wx in order to deal more conveniently with a minibatch of input at a time. (As an exercise, try working out the dimensions involved to see why this is so.) Finally, we apply the ReLU nonlinearity with the built-in tf.nn.relu activation function.

The remainder of the code for the fully connected layer is quite similar to that used for the logistic regression in the previous chapter. For completeness, we display the full code used to specify the network in Example 4-5. As a quick reminder, the full code for all models covered is available in the GitHub repo associated with this book. We strongly encourage you to try running the code for yourself.

Example 4-5. Defining the fully connected architecture

In [ ]:
with tf.name_scope("placeholders"):
  x = tf.placeholder(tf.float32, (None, d))
  y = tf.placeholder(tf.float32, (None,))
with tf.name_scope("hidden-layer"):
  W = tf.Variable(tf.random_normal((d, n_hidden)))
  b = tf.Variable(tf.random_normal((n_hidden,)))
  x_hidden = tf.nn.relu(tf.matmul(x, W) + b)
with tf.name_scope("output"):
  W = tf.Variable(tf.random_normal((n_hidden, 1)))
  b = tf.Variable(tf.random_normal((1,)))
  y_logit = tf.matmul(x_hidden, W) + b
  # the sigmoid gives the class probability of 1
  y_one_prob = tf.sigmoid(y_logit)
  # Rounding P(y=1) will give the correct prediction.
  y_pred = tf.round(y_one_prob)
with tf.name_scope("loss"):
  # Compute the cross-entropy term for each datapoint
  y_expand = tf.expand_dims(y, 1)
  entropy = tf.nn.sigmoid_cross_entropy_with_logits(logits=y_logit, labels=y_expand)
  # Sum all contributions
  l = tf.reduce_sum(entropy)
with tf.name_scope("optim"):
  train_op = tf.train.AdamOptimizer(learning_rate).minimize(l)
with tf.name_scope("summaries"):
  tf.summary.scalar("loss", l)
  merged = tf.summary.merge_all()

# Adding Dropout to a Hidden Layer

TensorFlow takes care of implementing dropout for us in the built-in primitive tf.nn.dropout(x, keep_prob), where keep_prob is the probability that any given node is kept. Recall from our earlier discussion that we want to turn on dropout when training and turn off dropout when making predictions. To handle this correctly, we will introduce a new placeholder for keep_prob, as shown in Example 4-6.

Example 4-6. Add a placeholder for dropout probability

In [ ]:
keep_prob = tf.placeholder(tf.float32)

During training, we pass in the desired value, often 0.5, but at test time we set keep_prob to 1.0 since we want predictions made with all learned nodes. With this setup, adding dropout to the fully connected network specified in the previous section is simply a single extra line of code (Example 4-7).

Example 4-7. Defining a hidden layer with dropout

In [ ]:
with tf.name_scope("hidden-layer"):
  W = tf.Variable(tf.random_normal((d, n_hidden)))
  b = tf.Variable(tf.random_normal((n_hidden,)))
  x_hidden = tf.nn.relu(tf.matmul(x, W) + b)
  # Apply dropout
  x_hidden = tf.nn.dropout(x_hidden, keep_prob)

# Implementing Minibatching

To implement minibatching, we need to pull out a minibatch’s worth of data each time we call sess.run. Luckily for us, our features and labels are already in NumPy arrays, and we can make use of NumPy’s convenient syntax for slicing portions of arrays (Example 4-8).

Example 4-8. Training on minibatches

In [ ]:
step = 0
for epoch in range(n_epochs):
  pos = 0
  while pos < N:
    batch_X = train_X[pos:pos+batch_size]
    batch_y = train_y[pos:pos+batch_size]
    feed_dict = {x: batch_X, y: batch_y, keep_prob: dropout_prob}
    _, summary, loss = sess.run([train_op, merged, l], feed_dict=feed_dict)
    print("epoch %d, step %d, loss: %f" % (epoch, step, loss))
    train_writer.add_summary(summary, step)
     step += 1
     pos += batch_size

# Evaluating Model Accuracy

To evaluate model accuracy, standard practice requires measuring the accuracy of the model on data not used for training (namely the validation set). However, the fact that the data is imbalanced makes this tricky. The classification accuracy metric we used in the previous chapter simply measures the fraction of datapoints that were labeled correctly. However, 95% of data in our dataset is labeled 0 and only 5% are labeled 1. As a result the all-0 model (which labels everything negative) would achieve 95% accuracy! This isn’t what we want.

A better choice would be to increase the weights of positive examples so that they count for more. For this purpose, we use the recommended per-example weights from MoleculeNet to compute a weighted classification accuracy where positive samples are weighted 19 times the weight of negative samples. Under this weighted accuracy, the all-0 model would have 50% accuracy, which seems much more reasonable.

ForI computing the weighted accuracy, we use the function accuracy_score(true, pred, sample_weight=given_sample_weight) from sklearn.metrics. This function has a keyword argument sample_weight, which lets us specify the desired weight for each datapoint. We use this function to compute the weighted metric on both the training and validation sets (Example 4-9).

Example 4-9. Computing a weighted accuracy

In [ ]:
train_weighted_score = accuracy_score(train_y,   train_y_pred, sample_weight=train_w)
print("Train Weighted Classification Accuracy:   %f" % train_weighted_score)
valid_weighted_score = accuracy_score(valid_y,   valid_y_pred, sample_weight=valid_w)
print("Valid Weighted Classification Accuracy:   %f" % valid_weighted_score)

While we could reimplement this function ourselves, sometimes it’s easier (and less error prone) to use standard functions from the Python data science infrastructure. Learning about this infrastructure and available functions is part of being a practicing data scientist. Now, we can train the model (for 10 epochs in the default setting) and gauge its accuracy:

`Train Weighted Classification Accuracy: 0.742045`

`Valid Weighted Classification Accuracy: 0.648828`

In Chapter 5, we will show you methods to systematically improve this accuracy and tune our fully connected model more carefully.

# Using TensorBoard to Track Model Convergence

Now that we have specified our model, let’s use TensorBoard to inspect the model. Let’s first check the graph structure in TensorBoard (Figure 4-10).

The graph looks similar to that for logistic regression, with the addition of a new hidden layer. Let’s expand the hidden layer to see what’s inside (Figure 4-11).

> **Figure 4-10: _Visualizing the computation graph for a fully connected network._**<br><img width="300" src="data:image/png;base64, ..." />

> **Figure 4-11: _Visualizing the expanded computation graph for a fully connected network._**<br><img width="300" src="data:image/png;base64, ..." />

You can see how the new trainable variables and the dropout operation are represented here. Everything looks to be in the right place. Let’s end now by looking at the loss curve over time (Figure 4-12).

> **Figure 4-12: _Visualizing the loss curve for a fully connected network._**<br><img width="300" src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAVMAAADVCAIAAABlt4RSAAAACXBIWXMAAAsSAAALEgHS3X78AAAgAElEQVR42u2dd3gc1bn/v7O9r3qxJblIlnA3hmtsE9xoptiYjlNMAtgQkkDiANcJNiUJgRBaSHLjBBxagPuY5AdJCCEhtEuMccDdcpHloi5t0fbdmZ32+2PWuyt5Vb1jq7yfx4+f0czsO2fPnu9pc97zMrIsgyCIUYaGsoAgSPkEQZDyCYIg5RMEQcofWhw8eLCsrKy1tXVwH6+rqysuLhYE4Zlnnpk2bRoVC4KUPzwYO3bs2rVrCwoKBvfxV155ZeXKlTqdjgoEMUoYCWVdFEW73b527drBfVyW5VdffXXz5s1UGghq8zOza9euL33pS1arddKkSS+++CIAv9/PMMyuXbuUG9avX79o0SIAb7/9dnFx8aOPPlpcXGy32++6666mpqYFCxaYzebp06dv3boVQDgcZhjmiSeeqKmpMZlMl1xySVtb2y233GKz2crKyp577jnFZjAYXLNmTX5+fm5u7sqVK/1+f/Kza9euLSsru//++3ft2sUwjMfjARCNRtesWZOTk1NRUfHQQw+JoqjIe/369WPGjMnJybnmmmva2tqSX2rLli1Go/Hcc8/t9mXD4fDq1atzc3NzcnJuvfXWcDjcUyb0dJIghi7yQCgvL7/77ruPHj26adMmnU63bds2n88HYOfOncoN999//8KFC2VZ/utf/wrgzjvv3LNnz6ZNmxiGKS0tff3113fu3Ll8+fLJkyfLshwKhQAsWLBg27Zt//znP0tLS51O5wMPPLB79+4HH3xQp9M1NzfLsrxy5cp58+bt3bt33759s2fPvuOOO5KfnT9//pYtW9ra2nbu3AnA7XbLsnzdddddcskle/fu/fDDD8eOHbtx40ZZlp9//vni4uItW7bs27fvggsuuPzyy5Nfas2aNT/5yU+U46effnrq1KnK8bJly84+++xt27Zt3bp1+vTpX/nKV3rKhJ5OEsSQZQDKD4VCDMP8/e9/V/585ZVX9u7d24vyDQaDIAjK+dmzZ99+++3K8b///W8AkUhEUe8nn3yinF+7dm1NTY1yLIqiwWB45513ZFnet29fe3u7cv6pp56aMWNGUvkff/yxcj6p/GPHjjEM09raqpz/+c9/fsEFF8iyfM8998ydO1eSJFmWDx06tGnTJuUGjuPy8vKOHz/eTfmHDh1SWnLl/BdffMEwzPHjxzNmQsaTVLaIocwAxvk2m+2+++675pprLr/88iuuuOL666+32WxK3zsjDMNotVrl2G63J6ffrFar0ic3mUwAkvNq6fdoNBqz2RyJRACUlpZu2LDh3Xff9Xg8PM9XVFQkH2EwGE4ej8iyPGnSJOVPQRCKi4sBrF69evPmzVOmTFmxYsV11113yy23KDe8/fbbU6dOHTduXDc7u3fvtlqtM2fOVP4855xzzGbz/v37L7vsspMzAUDGkwQxQsb5jz322J49e+bPn79x48bq6uqjR4+ehiTeeuutjY2NH330USAQePzxx/u832Qy7TrBvn37Pv74YwDV1dWHDx9+/PHH/X7/okWL1q1bp9z8hz/84Wtf+9rJRoxGY/pUvyzLkiRxHNdTJpyRnCGI06H8AwcOrF+/vrKycu3atZ999lleXt5rr71msVgYhknOfsXj8awn8f3337/tttvKy8uVMULvN0+ePJll2UAgUFVVVVVVNWHCBKXN/+Uvf7lly5Zly5b95je/2bRp01NPPSXLss/ne//996+//vqT7UyZMiUQCBw4cED5c/v27SzLTp8+PWMmZDxJZYsYygygt+9wOJ588kmNRvONb3yjvr6+ubm5pqbGYDBMmzbtqaeecjqdBw4ceP7552fMmJHdJE6aNOnXv/51SUlJbW3tY489lpOT08vNNTU1y5YtW7Vq1caNGwsLCzds2FBcXPyrX/3q8OHDv/zlL1944YWSkpJ33nln0qRJDMNs3rz54osvzmiwqqrq2muvvfnmm3/9618rs4ArVqyorKxsaWk5ORMy5gyVLWLkzO2/9957Z599ttFoLC8vT86Hb926ddq0aTabbdGiRbfeemtyhs9oNCY/uHDhwvvvv7/bbJwyS7d161bl/IMPPnj++ecnP+J0Ot944w1Zlvfs2TNnzhyr1bpo0aJf/OIX06ZNY1m222fT5/Y7Ozu/+tWvOhwOu91+0003eTweWZYjkcgdd9yRn59vtVqXLFlSW1sry/L555//5ptvpn/B9Ll9v9+/atUqu92em5u7evXqcDjcSyZkPEkQQxZm1Prn8zz/6KOPrlu37uRpQoIY8Yxe5RMEzfARBEHKJwiClE8QBCmfIAhSPkEQpHyCIEj5BEGQ8gmCIOUTBEHKJwiClE8QhPoMs713Ozs7WZZVjhX/+fnz52f9KSzLtra2Tpw4MeuW4/F4Q0NDcsugLCIIwpEjR9TwDlb2JjvrrLPU+EFra2unTp2qhuWDBw9WV1drNNlv2w4dOjRx4kS9Xp91y42NjTNnzjQajaT87uTl5aWSrtPZ7faSkpKsPyUWi3Ecp4bleDweDofVsCwIgt/vV8OyJElut1sNywDa2tpUsuxyuYqLi5P7wWURj8dTUlKihvI7Ozupt08QBCmfIAhSvuqEfCZ3I3wuygmClD+KYGQJAHiOsoIg5RMEQconCIKUTxAEKZ8gCFI+AKCurv4bX//WVcu/vPKm2z777Avl5Kdbtt1w/TcWL1r27W/d6/Ek1jC8+MJrV1x+w8UXXf3oT58WBJF+IYIYxsr/4Q9+vPLL1/35L69teODe/77voVgsFg6HN2z46foN97z3r7cmT6n52c+eAfDFFzvffOtvL7288c9/ea2xsflPf/wz/UIEMVyVL8vy8eONc+bMBjBlSo1Wq+lod2/d+sXkydUzZkzV6bQ333zTJ/+3NR6P/+u9j1asuLygIM9iMa9cee17//qIfiGCGK7KZxjm3HNn/fMfHwDYvn2X1WopKx/b2NBUMa5cucFms9nttra2joaG5nEViZMVFWVNjc30CxGEGpwmj51777trzerv/s//bOK4+GM/e1Cn07IsazSmwlqZTEaW5ViWNZqM6We62QkEAjzPJ49jsZjH48l6anmfLxQK+Xw+UWPOruV4PO73+9VIs+Kxo4ZlJcSgGpaVH1Ely36/3+v1quGrp+SGGh470Wh0RCmf4+Lf++4Pf/LI+nnz/quhoWnN6rsrKyeYzWa325P2nWMWi9lsNsdiscSZWMxsNp+snGTusCzL83zy/myqiGXj8TjLskK2jfM8z3GcKmkWhHg8roZlWZZVSjMAtS2roXwlnwVByH6Tc6JVGyHKP368kYvH5837LwDjxpVXVU3cs3vf+PEV//l8h3JDZ6ePZdnS0pLxEyqOH2tUTh45cnzixPHdTBUWFiaPXS5Xe3t7eXl51hPMGhhN2F9aWoqi8qwXGpZl1UizIAiRSEQNy5Ik+f1+NSwD8Hg8Kln2+XxlZWVqeOkGAoHy8nI12nwlQvTIGeePGVMSjURraw8C8Li9Bw8enlg54by55x6uO/L5f3YIgvi737544UULdTrtxRcvfvOtv3V0uMPh8Msvvb506YU0HiOI4drm2+22Rx7Z8OMfPR5jOa1G8807bznrrEkAHvnphp/97Bdeb+fMmdMeengdgLPPnn7jjVd//eY7RVG8dOmFVy67lH4hghiuygewYOH8BQu7b5s1d+65f/zTS91Orlp106pVN9EPQxDDvrc/XJFESLSIkCDljyYM/g542xAnF32ClD96hkD/frP0Xy/g3ZcpKwhS/uijqY7ygCDlEwRByh8l0FZ8BCl/9MCIJxZm0tw+QconCIKUTxDEMGaYxdWLRCJJf6ZwOMxxXDAYzPpTzMcPKH4eoVBIZgxZtByPxyORiBppFgQhGo2qYVmWZZUsA1DVcigUUsNXT0mzGh47yWixpPzuBIPBSCSCE7KMRCJqeHczc1ZMeOtJAJ2dnWJczqJlnufV88/3+XzDzj9fbctqKN/n83m9Xp1Op0adQsrPTGlpafLY5XJ5PB41Yl3HSkvx1pMAxo0bB2dBdtt8URTVSLPin6+GZUmSIpGIGpYBBAIBlSyHw+EJEyao4aUbjUYnTJgw3Nt8Guf3UNwNJsoEYgRDys8Ml1sCAG3HKCsIUv4orABilAcEKZ8gCFI+QRCkfIIgSPkEQZDyhyFcXikAtB+nrCBI+YOH5/mHH/7Zggsuv2zp9e+8855ycijH0pWNVgCIhamIEKT8wfP886+EQuF/vvf/nn32sddefYPj4kM8lq7MaKlwEKT8U+Xtv777ne+sMZlMk6or//Dq74xGwxCPpRsbUwUAx/ZTESFI+YMkFAp3dvo//fQ/K676yk033vLxx1sADJNYujIVEWJEojs9yhcEIRaNvfnWH/bu3f/tb937p//38uBi6XIcJ4pi8lgQBDXiMcZiMVapEwOe7NqPx+PqRdRUyfLwjajJsqwavnpKmimiZt/YbFZJkm64cQXDMDNmTJ0ypWbPntrBxdLt6OgIhxOzbpFIxO/3NzY2qvHTuuPaSgB+T3bt8zzf3t5usVjUUH5HR4fNZlND+R0dHQ6HQ42y4XK51PgFlaJis9nUUH5HR4fJZFLDS9fv948o5TscdovFHAyGk+VSq9UOLpZuRUVFeqHx+/01NTVqtPlmJtHPz679eDyu0WjUSLMgCLIsq2FZkqR4PK6GZQDRaFQlyxzHVVdXq+Gly/N8dXW1Gl66yf7sCBnnA7jkkiXPPfeSIIi1tQcPHDg8Y8bUYRBLt7waAPb/h8aEBI3zB8n31n7zgQ2PXrhkeV5e7o9//MO8vFwMl1i6fheVEoKUP+ihvu2ppx/pdnKox9Itm4SmOoq0Q4xIaPUuQZDyiXSKygDA3Uw5QZDyRxNGC0Db8hCk/NGGMx8Agl4IPGUGQcofNTjyASDgRWc7ZQZByicIgpQ/slEW8zQcpJwgSPkEQZDyCYIYhgyzuHqiKEqSpBwLgiCKohrOksIJtAVjmKY6qf24lKWnqJpmlSzLsqySZeUHVc+yKIqyLKuUZoZh1LBMys9MY2NjMuhyNBp1u90HD2Z/EM5xnKe50RAw5cXiuYC/tal9z66sRNrjeb6lpUWlQtPQ0KCGa5osyw0NDWq4pik/qMmkSghDJTfU8NJtaGiQZVkNL12v10vKz8yECROSxy6XKxwOT5s2LetPicVizQZdZY4ZgQYcRJ7ZkFc5PitBdePxuNFonDJlihptvlarVSM3JEmSZVkNy0pVqJJlSZKmTp2qRlUIoKamRo2qcP/+07f7G43ze6VQWcDbQjlBjDBI+b1SVA6A3PUIUv4ow2imPCBI+aMSRx4ANB+mqBsEKX80oUzsuZoQZykzCFL+CEfSnZi5LRwLAC7y0idI+aMiY068DaqalejtEwQpfxShTPLR/hwEKX/Q+P2BJYuX//FEnMyhHEs3hfJiz9VEZYUg5Q+Sp5/6n2R4mSEeS7cLyvR+2zEqLgQpf8B88cXO1tb2Cy6Yq/w5xGPpdpG9Mr3f0QR3M4Je8ByVG4KU3y94nv/548/ef//3k2eGSSxdAEDZJABoPw42ioAXPoq9QZDy+8cLv391wcLzx09IRcUbXCzdM4OyMw9N7xMjiNPhq9fQ0PTeex/94dXfpZ8cXCzdI0eOJOONsizb3t6+a9eurCeY4ziXy8VYGAC82W4MslWA1N5QX1cHQNQZuNZBelPyPN/a2hqPx7OeZsVLVyX//KNHjya3Rcgu9fX1avgsK0VFEAQ1vHSPHj3KsqwaXro+n6+ysnLkKP+TT7a63Z4rr7gRQDQa1Wq17W2uKVNqBhFLt7KyMrnXgsvlikajM2fOzHqCY7FYY2NjtVWTGOcHO/FpnibYOclpYoorYDTLig/fwInH4xaLRSUvXYPBoIbHqyzLGo1mxowZapQNSZLU+AUBMAwzbdo0NZSv1WpV8tI9cODAiGrzv/rVG7761RuU45899kxl1YTrrrsqEok+8siTn/9nx9mzZ6bH0l2//ifLll9mtZpffun1669fkfEXTT9Qo8VgGKa72aqZ2PEh01KP4opTeShzApXSrIZlWZZVspw5q7NnWdXioVKyR5TyM2K1WoZHLF0FZXo/4KHxIUHj/MHw3+u+mzwe6rF006EtOoiRBa3e7R+0hpcg5Y+avNEAJ+Jqpq/h5TkEvYgEKYcI6u2PRMZWAQAbTTX7XAxcDEYzAl4YzbA6EpciQYg8LA7o9JRtBLX5IwKTJTHI78V1JxJAgELuEqT8EYmyRYebtuggSPmjCmXAn3yxRy08QcofFSir95Mv9kSBsoQg5Y8CknvvB08s2m+qS83/EQQpf4T29s2JZn/fVsoMgpQ/mpi9GOjqrpve5/e7aP6PGC4Ms/f5x48fD4VCynE0Gu3o6Kitrc36UziOa29vT3q8auKsIeACoBEMEwC5o/HokSOJW5MHJ2ALA71Y5nm+ublZpdDODQ0NKnnsHD16VKXQlMeOHTMYDGpYPnr0KMMwavjqHTt2TBRFNbx0PR7PiPLSzSJlZWVJR3G32+33+6uqqrL+lFgsptfrU5bZKOMxJvv8DBeryLHK9rzMOinrLT2KZ74aaRYEQRAENSzLssyyrBqWAYTDYZUsx2KxqqoqNZTPcVxlZaUaXroqbYIwEpSfXtHq9XqNRmM0GtX4AQwGQ8qyBki2S0XlaKrT+9qRX5L5w1pNLyv5GIbpYjl7aLValSx3z42soqplg8GgRldFSbMaylepY0Xj/MGiTyuayp589bt7bnzpPT8xDCDlD5CqmQDQfJj89ghS/miiqByFZQh4e2v2CYKUPwKZNg8Amg5lvhoNUoefIOWPRJShfk+bcEcGrvxIEO5mcvgnSPlDvsOvuOgHvdkxKMTBRiHEKWsJUv7QRpnn+/CN/s7zuZszL+9jo2iqQ7ATAHhSPjHilH/sWMNtt35nwQWXX3vtqk8+Sax7Hx6xdDMydR4A1O/GX37br/vZaN++PZJIxZEYacq/794HFy9e8NHHb9/z/W//YN2PWJYdTrF0ARSWIacw9Wd5Na66HQAaD6F+15lMWDB7gw6ClJ9dBEG86aZrbrzpao1GM2/+HI2Gcbu9wyaWroLJ0mU9D4CqWQkHnp0f9bfPr8YSgICXNgIjhqjydTrttdctVxbe7tt3wGq1lpaWDKdYur33+RsP4cPNXc7HY5kjbWfcwy87lSspnxiSvX2Flpa2DesfeeCBe3U67XCKpdsTReWYdwUAHNndpcvdS6RtkigxNDh9HjuH647cc8+G73//2/Pmz8FgY+k2NzdHIhHlOBKJeL3ew4ezH9ya47i2trZubl6aOKv3d3S/tWxG4djD5pY64bWfe86/Nu4sThkJsEZXAwCuaJzRlQgTyjc2dbrc9Yws6wy6UKeGZ5lkXaDVCR1e0WQdUFIVy3yQi2t0TU1N/fd+YURBw4ah1ff5RFmWm5ubT/4tskJzc7PNZlPJstFoVMNXr6mpSavVquGl29nZOdK8dJubW++5Z8PDP/rBrFnTlTPjx1cMIpZufn5+Tk6Ocuz1em02W2lpadZTG4vFeJ7vbpmNanSZpt+XrpI3P6kL+Yp2/CO+9OvSCe9dKceukQoAQIqgoODElIcQj4RLnTbZka/RS91H/kazVDCwr6ORIgCk/GJBZwiHw/3PDSboZRiuP0+UZdnn86mRzwDcbrdKljs7O0tLS9VQvt/vLykpUcNXL9mqjRzl/+jhx7/9nTVJ2QM4b+65g4ilm97yRKNRo9GoRouh1WotFkt3yzoNYpYMd1ssWHIDPnxD42kx/fEpfO1+OPIBgA3C0v1+nueNRqPVaoXNhqgFWqab8tH/r8NzkKTEI2w2QWcwm80DyA2Rhcj154mSJA3M8kBQ1bLValXD71VJsxrKV2mTkjOm/JaWtu3bd+3ZU7th/SPKmUcffWDxkguGUyxdAAYj8koQ8GTYdbdqFowWfPgG3M14bj2W3oypc/uwFjzlCXmf64z5CyqVjt4AjRYEjfN7YuzY0u07Pjr5/HCKpQtAo4XVAa0One0ZxF9ejRV34N2X0VSHjzajcGxir96eCPT7JXwkiGgQFkcqmNcZR6l0CstgspCEhim0eneAmCww25Bx9OjIxw3fQ9VMsDG88TSeX99jmxzs7OMpAp/oEUSCCHi6rOpno4nVO+ldBu70bv5NexOQ8kcjuUWJeDsZWXw9ALAxBLz4y28HJhIuhqY6dLYjEkDbMQQ8EOKp/oUSz5OLJlbvpPc7+ukgGAlmYZRxisMEd3OPrzwJUv5Qx5YDR+YdOOHIx/d/g6U3A0Djof4u7O+mT4V0bXMxuJrgakKczfARUeiXnjvbuy/4O821gCiCjWZe5kSQ8odHn99Z0NsNU+di3hUwmtF4CG88g9rPEi12f9vGeHdZJj976lF9ZAlcLFGt9C9GmJbn0FSXhTWIp6J5nwvuZqo1soWOsmDw6A29udbOvxLOArz7EhoPofGQMuGntedqJi8CgPpdKK+BsYflMYrfniQOIHqft9XsaUKgBMrKHMWy4hqsRAFPVhlxtstEg8D3sllwr2OTaL9m+CJBiDwsDuj0p+SPyHPgYhBF6KnkkfLPcIepr3daU+eidiua6oDEon2Nq2mMtx17ctF4CCYzqmbCUYDySSgs71ILKC08Hx+A035yq3alZS4ZB72xSwchOQvYbX6xH8pnTmXXkEgAXAwGc+opg2u3B11rsFG4m2E09/G2hZRPZJOr7sCHmzF7CTY/DUc+3M0Gvwt+FwCwMez7LHWnMx/nXIgpc3vsCPSH5CxAetuovIbsqRKJhRHqBID8UohCd5dEpYoTszod0M94EgKPaBBafeJ1ppJ+nqNXiaT8M43VCaO5j1d0RnNitu/bTwGQ/v0X4eAXBjaM+Vci4AEXQ/3uRAsf8OKDzdi3FY48OPMx78rBVAHJRj4aTO0C1PsOf2F/4iDgRdjfpW1U3JBsuVo2kjKV7Dv0c9pi0G21wCPghdHcZSHDoK0NrqPhahqpKxdI+aeifEeGznM32Qt8+lhdPO+y5jHTJk6c2KVsOQvgagIXxbsvJybwAdRuxfXfG3wHdRD7eSpVgPISwZGPeCyx3MiWm5J616+T6E1YHABSg3mkrT4ymhNttSwNdIhxSvhchoCLESZBa05o/lQCVw2ioyGJ4OPQaDL2oUj5I52icgQ8fXQKFG0r8blX16B+d2JqQFkOVFYNABU1CHgQ8GLpzdjxAVxNcObDaIHJAkce7HlZHsHyHLhoMtlMKC393V40JucjZAmShIAXjjw4CxJ7ihpMqQVIcRZmWxfl+zoSCuFiXRSSPit5ciPfzylPntPyHETxlLoJp9LRiHNwN0Orw5iJpPws0N7eHo0mepuhUMjn8x07dizrT2FZtr29vZ/uE9pgUCPymkydSU7Q6yJ+bSQVXVcQBI/HYzKZejSXW4EvVWjDvtxtfzW4GhLxPE5E9ZB/90Mm0/t82WCKjZ8etFd4Qh3xovGnngOiyZfs4cstLR6Px2rtl/uwGIiI7W4tG9ayEbEzAK1O+fpiIILmZiYW1ogCACEc1wU9AEINja1COJFdReMAKK7NfGdIGw1o4iwALiYDMLpaAUh6r+j2aQRO0hklQ5dsZIS4LuyTNDrRka/3tXs8Ht3xYzDbkj8BF9f0OSmr+GJLeiOfWwLA4G5kZFkMRITOUPKe1tZWg8HQu8dO0qeb4wYQ3djn85HyM2Oz2ZIu6LIsm83mpNNuFonFYpFIpL+Wc3IA6NqOnnzFnJOj0UoaTaqfyfO8xWKx2+192LTb4xeu1P3zZQCy0aT1tkl6oybsZ+IsjGahdCIATcinCfvAxZTwvva6z8/G5wDEsZVS/hh27rKUJEKdPUX+7Q29PT2f+06z0jqaLZpYGHoN9HaZYcAwjN0OQNYwjCTCcmLmQuZgtwOwGQ1Jy2YlJ2OdAESnUytGYdR3Oy8bTLLBoAlHZZ0MLsBIIgDJliPZ85hoSMuHZYNJdNi1fMhsNudYTHCmfgKL1SInR2HdhBrwMCIv2vMYs1ErRmWdXkx7qGS2SGmFwWaz5eTkZFA+zzFcTDZZodMzXEwrRpOJJ+VnQfnpKjKZTLm5uVl/islkCoVCA7Msj4VW16VjrzcgNxcaEYw4YOUDsNux8p7kfIGGi6HpEFzNqJqpS/btuViiENfvkvZ+qjm6F4C25Yi25Yje70LAA2cBlE1Bzl6CuDKbGAUbAwCTGUYLjBZMm4fKGQnP4ozfTJatVms/lZ9Ief+nStItG7SIBhMfz8kBH85wXm+AJCWODSdKr8MJZy40IiQ7jGYYtDCbrFar0+nUpP8EOTmJoYTVDL878VllnMWHwclwOBLHAJRfP2xP1BR2G2RJGZI4HI7c3NyE8nkOfjd0BuQWwdUE8LCYYHUgogVvBwC7rf8zGm1tbaT84UZeSWJmKzkQ7aFjKRtMKCzLvP3+yROE6cdVs1A1K/MNVbOk8dOaDuwZV1yI2q3Y8SEaDwFpHoFb/3bSkCYGNgZ48UETPtiMonKYLDCYMO8KOAsSlrkYAp7UkJuLwd0EAIXlCHiy/3q8sz11nJ4/XNqe5T29m4wEU28004flydWKSHuLkZR9+ryDQsaNjAUekQCCnXDkgYuZPU2MVYO8IjgLEuuRjXLazfHU/6dhLpOUPySwOvr2w0vNEOhgtiEaPKVp5/TyaXGiqBxF5XAWYPv7mDoP7mYUlqF2a2IqbukqGC1w5qN+d8rv4MM3EOxMrcyt3w3nifY/4IXJjLOXTNjzb/wlkOGRFTWYvRhGMxz5MFpOaSVCL/T5nkK5QVF4t1XS6SuOY+HMs3d8mltUxpd/opC4oZcfl41mfs3Z/1WYpPxhT/orcWcBnAWZZ/h1euQWweZMFJqM5TJjfdFnYZq9BLOXpP6cf2WXoYFyQ5KqWQh6Ub8bAQ+aDsPd3GXvADbGbP1bqpQ48hJfxGgGF0ssTE6MkcxYdIdd9BEAAA4XSURBVAOaD8FRgPrdMFkwZS6MJrhbwEVRvxsBb+KVhGKnahaqZmp4NgsZ3lOG9Ed1sUiqr5He0VBeTPZe+/Ac4rE+njVUg6aR8k9PNqdeEwhmO3R6OPISJ/VG6I3QaPulfEdel+7rQOujHs3mJ+oCLoZ3X0JhGYrK4GqGyYJ9W8FFO0tr8uYsTlRkrqbE2NjVhB0fpFYisTG823WflcaTwg0HvIA38dn63QAmAfgHMG0uiirARsFF4W5B2STMXoL9nyXObP8AJjOKKhIzF4VjUTYJJkvfHY1IEH3uwxfwZD4fDfbhHCUKqW2R0jsaogB3M/g4Pn0bLfWYOg+Xfo2UP1oxW1FYBq1Wlhmxvh46fXdXP6ujyyhXeb99cu/RaIHOAKMZkSBMloEt17E6+r7faMZVd6T6AgBmL5Fl2Vtfn5fsxSiyB1BUjqU3J7oSQS8+fRu1nyWuFo5NuCcazZg6F0YLnAXgonDkwZEPLpZYs5DsB+37DPisS5Wx4/3ETGRyViI5c9F4CNs/SHU05i+DqxFGC9wtcORiyjyl+ijsaNV8/FKi/2Iyg43BmY/ySXAUJKYzq2bCaO6xGq3fBYMZzoLU1Ea3KiYWhjaTfGLhxPDtyB64mlAwhtr8UYxGm1gEFu+r72fLgShAEhPjBZ8rNS+l0UB7wo7O0Fs3UquDRtNlMkyrQ14JLI5+zSwOoivhyMfSmxPrlBNDjGWJybmMbw1O9BqOtrRPNIj4cHNiiKSMCOp3g42hsAxVMxH0omommupQNStRxdR+hqa6VEfjg65hTk64QuR0m87EiXhESZz5KCxDnAUbRdCDsmpUzUL9rsSZk12SK2oAwGAq4gWU3IxoAEZLohuSrBRcTdj6t0SNplhwt5DyR9M4f/C9AxtMltQ8U25RSvnGtAhfyaXsJ6+oVa4aLV1Erkwva7UwmqHTD6yzYMuRB7HTppIJvWdFUTnvi6KmpvuGpco7hfR3B8mXGkXlqWMuhk//iqbDiSlJR156R6MzwubWTGcYJjGp6chH7WdwNcLdkhhEdKsI6ncnV0x1r6SUaZoTgxc7gI33ZagUTh7dDOUBKOl0KJK+2FuZS8spzLB0XBky9LlAGIDVmTCryKkX5ZdXJ9xUkhhMcWfh4CvBgVY03aZIe79t8Q1dzqT96T18OKeykkkf50+dm6piuBi2vw8ARWWJ3FNkXzYp0R9R8iGZDGVswkUBxHd8ZAi4M89oGBXP63zs+ADl1ajfDXcT2OgQdPhRV/mfbtn2zDMb3W7P1KlnPfTwDwoK8vq8+uILr73xxlvxOL9kyYJ777tLpxv1+zrnFkEUYTAObItrgzExD6/RQJJgtg62DjLAbJW0epSMgygi6B3YzoJF5WCjg3EfOg39MuWVR5L09x3JxHcbngAAWizFFSaNtnR8YixTvytRd0ydl+rjnHMhjGY8+c0usxVDagCqnumMcbJ7vzp0o2if2fbfZOmP7CW9EQVjUFgGqxMaLSx2mG3IKUJ5dfeP6w2p//safciMJpUMe25vM4g5XbsGyhIgkwXl1cgpRP/D3Wh1KCxDeTVsOWqtETgFJL1JLpuUWMWgjESU16jpSXU14fn1ieN1y/D7B/v71nYEKD9jnOzerw7dKNqnDcNgnTo1mrgtLzFNoIzqlYog4y79JeNRXo2S8V0qAo0mUXZzi2B1wGiG2db9HQSj6fIRozlVfWh1qUGKRtO9WrHnpq6eXO/kFHb5U6dPdI9zi1BU3t1Utxqk259WB8qrMbYy7iwacIQCjSaRCadY3QS9+MtGBLyAsrZPxt4t+P2Do6W3nzFO9rgTZ3qKoj1nzjmJSZOhH0U7Qx/SAgd625O778I32NGN3ihrhUHUF5AkOAugN6YWmeqNyCtBJNjjGEFZfch3phJstsFoSQwxAOgMvaku+RH+xKIgey4EPvEyQglnkk5OUZfZSr0xsfAuOSOg7HemfB2lHdZoRb0R9lzE2S7Tn8qkSbKOSJ/y0BtSVSGA9uNdXo6kfVBmGNlggiR0yUZbTmLmJeTDp2937eQziXmEzvbEKu+RrfyMcbJ7v9pnFG2Px5MMthsMBgOBQEtLixopd7lcg4/3FuaAzE5XPM+73e6MadZG44wkCgP/OppYWBMLxfXmniz31q0ORjVsWOA1csZWzh8CIMuyx+NJWdZYIYPxh7TBoKxjJYsdokaGAC79K3PKZ7umM8oou3qFWUaISzyjiQQ9oYiN16ClheFiGj4u6U1yTAREhNO+iMBroywjxGVGw8iSrGMZyAwXk8w2SdLLLS06fxAAJFHMKZI7XMmi0pKbq+N4ho0wJ/YFkayM1ueSdXrZZBUlVsNzmlgIksgIvJhbLKXnngDloQm1Kw+NhmSLvZ2VdLzGHEpUGaI1B4AMDgDCLZDEXJ83re6XwSTE76mr5cp7c/UPBoMjQfkZ42T3frXPKNo6nS7ppavX63U6nRpBCCVJ0uv1KoU37DHNecUABvNIQx6ceTpB0Ht8A05zQSnQx2a2sixnSLPBAIsNgHZA6ew2zrTnxD2hhGWDAXBmtmYwwJKh96FNPr1knKJ8TVqPSUmzxmSCM6/bQ5kTHwescOYxXEzjdzH2nP48lAHgDRkMBo3y0IwD5qIyHGJSrX2ys6LXy73+QGpE5j4Dys8YJ7v3q31G0e7mM2+1WgsLC7Oe8lgsFo1G1bAcj8cDgYAalgVB8Hq9aliWJCknJ0cNy8oPqpLltra2goKCfsXSlUQ47b1NXnalo6OjsLCwt505yjLtw2OyOs9Z0Ltlt9s9Emb4zpt77uG6I5//Z4cgiMk42eFw+M9vvdPT1YsvXvzmW3/r6HCHw+GXX3p96dILQRCqi0Cb5Wilcy7NsAnXwmtGywyf1Wo5OU62x935k588cdWKyzNeHdJRtImRLf7scu/v8PGfUL8bsTDGVmL6l1A1c7QoH5niZI+fUPH5Fx/0dBVDOYo2QQyIhddi4bVDt66jH4ggRmMvh7KAIEj5BEGQ8gmCIOUTBEHKJwiClE8QBCmfIAhSPkEQQ45htg+fz+dLbu/h8/lEUayvr8/6UwRBCAaDaliWJEk9y6FQSA3LAMLhsEqWI5GIemk+evQowzBZtxwKhY4dO6bRaNTIDVJ+j+Wb5xMh0AwGQ0lJiSiKajzFYDAMO8smk0kNy7IsG41GNSwDMJvNKllWckMN5Su5Icty1i2r5Bg+EpSfn5+fXqnzPF9TU5P1p0Sj0ePHj6thmeO4w4cPq2GZ5/n9+/erYVmSpN27d6thGcD27dtVsrxz587q6mo1WuY9e/ZUVVX15qU7WPbt20fjfIIgSPkEQZDyCYI4RRg1JioIgqA2nyAIUj5BEKR8giBI+QRBkPJVoLm5dc3q7y5aeOXKm27btWtvf65+umXbDdd/Y/GiZd/+1r0eTyJY9YsvvHbF5TdcfNHVj/70aUEQVU1zOBy5/4c/XrJ4+aWXXvv737866FSdzjQnWbP6uz9Y93Av2Tuk0vzaa3+8+KKrFy288pmnf6PMfPe/SCTp/eoQQh5N3HbrXX94ZbMgCFv+/dkll1wTj8d7vxoKhZYsXr579z6eF371q+fuuWeDLMuff75j+fIvu93eSCR6x+1r//f1P6ma5sceffq+ex+MxdjW1raLL7p6+/Zdg0jVaU6zwp///M4Vl9+47r8f6il7h1Sa/7Nt+1XLv9ze7vL5/Lff/r39+w/1v0gk6f3qkGIUtfmdnb6DB+tuvOkarVY7//zzCvLzdu3a1/vVoRDw97y559519+0mk7G0tGTqtLMaGpoGkarTH6TY7w+88PtXb/76yl6yd0il+a9vv7tq1U3FxYU5Oc6NG5+aPLm6/0UiaaT3q9TbPzM0NjaPGVOi0yViKnSL1Zvxak8Bf8dVlGc0ogaLFn1p7NhSAJFIdH/twVmzpg8iVac5zQCeefo3X//6l3Nznb1k75BKc13dEX8g+JUvr77qqq88/9zLAyoSqVLU61VS/pmBZblkNE4AJpOJZdnerw4u4K9KiV+37uEVV185YcK4QaTqNKd5+/ZdzS2ty6+6rO/sHTJpDoXC+/cffH7Ts5uef/btt//xwfv/1/8i0SXNPV8l5Z8ZzGZTLJaSejQaM3eJ7ZvhqtlsZrue7E/AXzUK5bfuvGfy5OpvfvMWAINI1elMM8/zP3/82R/+cG26h2xP2TtE0gzAbrNeeeWlZrO5oDD/iisv/fyLnf0vEulp7uUqKf/MMG5ceWtre9K9/8iRY+mxejNeHT++4tjxxuRItZ8Bf7MLx8XvvmvdRRcvuvPOW5Uzg0jV6UxzXd2R1tb229d87+KLrv7xj5746KMtq2+7O3P2Dpk0AygpLQkGQ8k/dVpt/4tEKs29XiXlnxlycpzTZ0x5+aX/FUXx3Xff5zhu2rTJAP7+9395vZ0Zrw6FgL8vvfhadXXlypXXps/59TNVx481btmyDcDpTPPUqWf93yfvvPevN9/715sbHrhn0aLzn3v+Fxmzd+ikGcAllyx+/bU/BQLBzk7fO3/755zzzul/kUimOePVIaqHUfVWr6Wl7bZb71pwwRVf+fLq/bUHlZMXXbhix449PV3duvXza69ZtWjhlXfftc7n8ysnX3rp9aWXXnfxRVc/8cSvlO1Z1OPyy66f819LzptzkfLviZ//sv+peuONt26//XunP81J3nvvw+RbvYzZO6TS/Oyzv12yePmll1z729++2EuaT8789DRn/GmGIOSrRxCjEVq9SxCkfIIgSPkEQZDyCYIg5RMEQconCIKUTxAEKZ8gCFI+QRCkfIIgSPkEQZDyCYIg5RMEkU3+Pzku7mERBnZ+AAAAAElFTkSuQmCC" />

The loss curve trends down as we saw in the previous section. But, let’s zoom in to see what this loss looks like up close (Figure 4-13).

> **Figure 4-13: _Zooming in on a section of the loss curve._**<br><img width="300" src="data:image/png;base64, ..." />

Note that loss looks much bumpier! This is one of the prices of using minibatch training. We no longer have the beautiful, smooth loss curves that we saw in the previous sections.